In [1]:
#Loading libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
import re
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
import warnings
warnings.filterwarnings("ignore")

In C:\Users\Mirna Elizondo\anaconda3\envs\condaEnv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\Mirna Elizondo\anaconda3\envs\condaEnv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Mirna Elizondo\anaconda3\envs\condaEnv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In C:\Users\Mirna Elizondo\anaconda3\envs\condaEnv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\Mirna Elizondo\anaconda3\envs\condaEnv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was depr

In [2]:
df = pd.read_csv('../data/processed_diabetes_data.csv').drop(columns=["Unnamed: 0"], axis=1)

In [3]:
df.shape

(101763, 38)

In [4]:
readmit = df['readmitted'].value_counts()

# Print the counts
print("Occurrences of Readmitted:")
print("Value 0:", readmit[0])
print("Value 1:", readmit[1])
print('%:',readmit[1]/readmit[0])

Occurrences of Readmitted:
Value 0: 90406
Value 1: 11357
%: 0.12562219321726434


In [5]:
df.columns = [re.sub(r'[^\w\s]', '', col) for col in df.columns]
df.columns = [re.sub(r'[_]', ' ', col) for col in df.columns]
df.dtypes.unique()

array([dtype('float64'), dtype('O')], dtype=object)

In [6]:
numerical_columns = df.select_dtypes(include=['float64']).astype(int).columns.tolist()
object_columns = df.select_dtypes(include=['object', 'O']).columns.tolist()
print("Numerical Columns:")
print(numerical_columns)

print("\nObject (Categorical) Columns:")
print(object_columns)

Numerical Columns:
['encounter id', 'patient nbr', 'age', 'time in hospital', 'num lab procedures', 'num procedures', 'num medications', 'number outpatient', 'number emergency', 'number inpatient', 'number diagnoses', 'readmitted']

Object (Categorical) Columns:
['race', 'gender', 'weight', 'admission type id', 'discharge disposition id', 'admission source id', 'diag 1', 'diag 2', 'diag 3', 'max glu serum', 'A1Cresult', 'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburidemetformin', 'change', 'diabetesMed']


In [7]:
df = pd.get_dummies(df, columns= object_columns, dtype=int).fillna(0)

In [8]:
df.to_csv('../data/diabetes_data_encoded.csv')

In [9]:
X = df.drop('readmitted', axis=1)
y = df['readmitted'].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
rf = RandomForestClassifier(n_estimators=51)
model = rf.fit(X_train, y_train)


rfe = RFE(model, n_features_to_select=100)
rfe.fit(X_train, y_train)
selected_featuresRFE_reg = X.columns[rfe.support_]
print('Selected Features:', len(selected_featuresRFE_reg))
print(selected_featuresRFE_reg)

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
rf = RandomForestClassifier(n_estimators=51)
model = rf.fit(X_train_smote, y_train_smote)


rfe = RFE(model, n_features_to_select=100)
rfe.fit(X_train_smote, y_train_smote)
selected_featuresSMOTE = X.columns[rfe.support_]
print('Selected Features:', len(selected_featuresSMOTE))
print(selected_featuresSMOTE)

In [ ]:
under_sampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = under_sampler.fit_resample(X_train, y_train)

In [ ]:
rf = RandomForestClassifier(n_estimators=51)
model = rf.fit(X_train_under, y_train_under)


rfe = RFE(model, n_features_to_select=100)
rfe.fit(X_train_under, y_train_under)
selected_featuresUnder = X.columns[rfe.support_]
print('Selected Features:', len(selected_featuresUnder))
print(selected_featuresUnder)

In [ ]:
pipeline = make_pipeline(SMOTE(random_state=42), RandomUnderSampler(random_state=42))
X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)

In [ ]:
rf = RandomForestClassifier(n_estimators=51)
model = rf.fit(X_train_resampled, y_train_resampled)


rfe = RFE(model, n_features_to_select=100)
rfe.fit(X_train_resampled,y_train_resampledy)
selected_featuresResampled = X.columns[rfe.support_]
print('Selected Features:', len(selected_featuresResampled))
print(selected_featuresResampled)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define LightGBM parameters
hyper  = {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 50}

# Create and fit a LightGBM model
light = lgb.LGBMClassifier(**hyper)
light.fit(X_train, y_train)

# Feature selection using SelectFromModel
threshold = 'median'
feature_selector = SelectFromModel(light, threshold=threshold)
model = feature_selector.fit(X_train, y_train)

# Get the selected features
X_train_selected = model.transform(X_train)
selected_features_mask = feature_selector.get_support()

# Get feature importances
feature_importance = light.feature_importances_
feature_names = X.columns
# Create a DataFrame for feature importances
print(feature_names, feature_importance)

In [ ]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Define LightGBM parameters
hyper  = {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 50}

# Create and fit a LightGBM model
light = lgb.LGBMClassifier(**hyper)
light.fit(X_train, y_train)

# Feature selection using SelectFromModel
threshold = 'median'
feature_selector = SelectFromModel(light, threshold=threshold)
model = feature_selector.fit(X_train, y_train)

# Get the selected features
X_train_selected = model.transform(X_train)
selected_features_mask = feature_selector.get_support()

# Get feature importances
feature_importance = light.feature_importances_
feature_names = X.columns
# Create a DataFrame for feature importances
print(feature_names, feature_importance)

In [ ]:
under_sampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = under_sampler.fit_resample(X_train, y_train)

In [ ]:
# Define LightGBM parameters
hyper  = {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 50}

# Create and fit a LightGBM model
light = lgb.LGBMClassifier(**hyper)
light.fit(X_train, y_train)

# Feature selection using SelectFromModel
threshold = 'median'
feature_selector = SelectFromModel(light, threshold=threshold)
model = feature_selector.fit(X_train, y_train)

# Get the selected features
X_train_selected = model.transform(X_train)
selected_features_mask = feature_selector.get_support()

# Get feature importances
feature_importance = light.feature_importances_
feature_names = X.columns
# Create a DataFrame for feature importances
print(feature_names, feature_importance)

In [ ]:
pipeline = make_pipeline(SMOTE(random_state=42), RandomUnderSampler(random_state=42))
X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)

In [ ]:
# Define LightGBM parameters
hyper  = {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 50}

# Create and fit a LightGBM model
light = lgb.LGBMClassifier(**hyper)
light.fit(X_train, y_train)

# Feature selection using SelectFromModel
threshold = 'median'
feature_selector = SelectFromModel(light, threshold=threshold)
model = feature_selector.fit(X_train, y_train)

# Get the selected features
X_train_selected = model.transform(X_train)
selected_features_mask = feature_selector.get_support()

# Get feature importances
feature_importance = light.feature_importances_
feature_names = X.columns
# Create a DataFrame for feature importances
print(feature_names, feature_importance)